In [13]:
%matplotlib inline
import os
import glob
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

In [2]:
url = 'https://worksheets.codalab.org/rest/bundles/0x9d8c9e97326746459b4eb6564324d444/contents/blob/lm_results.csv'
df = pd.read_csv(url, header=0, index_col=None)
print(df.shape)
df.head(n=3)

(160, 179)


,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,AutoApprovalDelayInSeconds,Expiration,NumberOfSimilarHITs,LifetimeInSeconds,AssignmentId,WorkerId,AssignmentStatus,AcceptTime,SubmitTime,AutoApprovalTime,ApprovalTime,RejectionTime,RequesterFeedback,WorkTimeInSeconds,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.IDX0,Input.CTX0,Input.TYPE0,Input.OUT0,Input.LOGP0,Input.IDX1,Input.CTX1,Input.TYPE1,Input.OUT1,Input.LOGP1,Input.IDX2,Input.CTX2,Input.TYPE2,Input.OUT2,Input.LOGP2,Input.IDX3,Input.CTX3,Input.TYPE3,Input.OUT3,Input.LOGP3,Input.IDX4,Input.CTX4,Input.TYPE4,Input.OUT4,Input.LOGP4,Input.IDX5,Input.CTX5,Input.TYPE5,Input.OUT5,Input.LOGP5,Input.IDX6,Input.CTX6,Input.TYPE6,Input.OUT6,Input.LOGP6,Input.IDX7,Input.CTX7,Input.TYPE7,Input.OUT7,Input.LOGP7,Input.IDX8,Input.CTX8,Input.TYPE8,Input.OUT8,Input.LOGP8,Input.IDX9,Input.CTX9,Input.TYPE9,Input.OUT9,Input.LOGP9,Input.IDX10,Input.CTX10,Input.TYPE10,Input.OUT10,Input.LOGP10,Input.IDX11,Input.CTX11,Input.TYPE11,Input.OUT11,Input.LOGP11,Input.IDX12,Input.CTX12,Input.TYPE12,Input.OUT12,Input.LOGP12,Input.IDX13,Input.CTX13,Input.TYPE13,Input.OUT13,Input.LOGP13,Input.IDX14,Input.CTX14,Input.TYPE14,Input.OUT14,Input.LOGP14,Input.IDX15,Input.CTX15,Input.TYPE15,Input.OUT15,Input.LOGP15,Input.IDX16,Input.CTX16,Input.TYPE16,Input.OUT16,Input.LOGP16,Input.IDX17,Input.CTX17,Input.TYPE17,Input.OUT17,Input.LOGP17,Input.IDX18,Input.CTX18,Input.TYPE18,Input.OUT18,Input.LOGP18,Input.IDX19,Input.CTX19,Input.TYPE19,Input.OUT19,Input.LOGP19,Input.IDX20,Input.CTX20,Input.TYPE20,Input.OUT20,Input.LOGP20,Input.IDX21,Input.CTX21,Input.TYPE21,Input.OUT21,Input.LOGP21,Input.IDX22,Input.CTX22,Input.TYPE22,Input.OUT22,Input.LOGP22,Input.IDX23,Input.CTX23,Input.TYPE23,Input.OUT23,Input.LOGP23,Input.IDX24,Input.CTX24,Input.TYPE24,Input.OUT24,Input.LOGP24,Answer.0,Answer.1,Answer.10,Answer.11,Answer.12,Answer.13,Answer.14,Answer.15,Answer.16,Answer.17,Answer.18,Answer.19,Answer.2,Answer.20,Answer.21,Answer.22,Answer.23,Answer.24,Answer.3,Answer.4,Answer.5,Answer.6,Answer.7,Answer.8,Answer.9,Approve,Reject
0,3TCFMTM8HEICE4NA2ANQOLZRG1C219,3KGK41XE0IFLTI1OFAGBTFLBSPEWN4,Estimate Typicality of a News Story,Check whether a sentence from a news story is ...,"survey, NLP, probability",$1.00,Fri Nov 16 17:27:42 PST 2018,40,BatchId:3439876;OriginalHitTemplateId:920937332;,3600,259200,Fri Nov 23 17:27:42 PST 2018,NaN,NaN,30X31N5D64K4ZBF1IB0QEO0KTOBSA5,AFUO1VYEFRMO4,Submitted,Fri Nov 16 17:28:35 PST 2018,Fri Nov 16 17:37:19 PST 2018,Mon Nov 19 17:37:19 PST 2018,NaN,NaN,NaN,524,0% (0/0),0% (0/0),0% (0/0),0,0,LM,London attended as their candidate for the 201...,-79.713061,1,1,LM,"NOBODY doubts that spring is here , starting w...",-53.363786,2,2,LM,' If you want problems with our phone service ...,-52.835088,3,3,LM,Mr Marchesani and his family set up a small bu...,-115.084019,4,4,LM,The White House said it is aware of the incide...,-24.52095,5,5,LM,"Prairie View A & M ( 0-1 ) , coming off a bye ...",-76.706032,6,6,LM,"Households ' land inflation , which closed on ...",-135.713855,7,7,LM,Both Democratic candidates backed FCC regulati...,-74.051955,8,8,LM,The GOP name for 2-2 February has just moved o...,-114.770884,9,9,LM,"' The task is much tougher yet , ' he said .",-31.71697,10,10,LM,He says police used handheld electronic commun...,-69.278525,11,11,LM,' I don 't think it is useful to know ' whethe...,-98.811419,12,12,LM,His three seasons dealing with a hierarchical ...,-83.558109,13,13,LM,"But , fortunately , this experience has not ov...",-46.943354,14,14,LM,ATLANTA - Killings dropped significantly in th...,-113.479394,15,15,LM,The means that were taken to sell the medicine...,-73.262174,16,16,LM,The American trip landed in Prague for a Vienn...,-114.573159,17,17,LM,' Developing countries are the most vulnerable...,-125.392514,18,18,LM,That was before the aides were forced out when...,-143.883346,19,19,LM,Elated M

In [3]:
responses = ["Invalid", "Rare", "Specific", "Average", "Typical", "Very Typical"]
response_dict = dict(zip(responses, range(len(responses))))

In [4]:
worker_counter = Counter(df['WorkerId'].values)
print(len(worker_counter))
worker_dict = dict(zip(sorted(worker_counter), range(len(worker_counter))))

124


In [5]:
ctx_counter = Counter()
for i in range(25):
    ctx_counter.update(df['Input.CTX' + str(i)].values)
print(len(ctx_counter))
ctx_dict = dict(zip(sorted(ctx_counter), range(len(ctx_counter))))

50


In [6]:
model_counter = Counter()
for i in range(25):
    model_counter.update(df['Input.TYPE' + str(i)].values)

print(model_counter)
model_dict = dict(zip(sorted(model_counter), range(len(model_counter))))

Counter({'LM': 2000, 'HUMAN_LM': 2000})


In [7]:
combo_counter = defaultdict(Counter)
for i in df.index:
    for j in range(25):
        combo_counter[df.loc[i, 'Input.CTX' + str(j)]].update([(df.loc[i, 'Input.TYPE' + str(j)], df.loc[i, 'Input.OUT' + str(j)])])
print(len(combo_counter))

50


In [8]:
for key, counter in combo_counter.items():
    print(key)
    for pair, count in counter.items():
        print(pair, count)
    print()

0
('LM', 'London attended as their candidate for the 2016 Summer Olympics ( August 31 -- Thursday , Monday ) .') 40
('HUMAN_LM', 'Having a little flexibility on that issue would go a long way to putting together a final package .') 40

1
('LM', 'NOBODY doubts that spring is here , starting with the French Alps .') 40
('HUMAN_LM', "Long before the advent of e-commerce , Wal-Mart 's founder Sam Walton set out his vision for a successful retail operation : ' We let folks know we 're interested in them and that they 're vital to us-- ' cause they are , ' he said .") 40

2
('LM', "' If you want problems with our phone service , take them to the bank .") 40
('HUMAN_LM', 'A spokesman said the company has been affected by the credit crunch in the United States .') 40

3
('LM', 'Mr Marchesani and his family set up a small business to plant in the ground general , delivering a less-than-perfect product line to everyone .') 40
('HUMAN_LM', 'Abu Dhabi is going ahead to build solar city and no poll

In [9]:
combo_counter = defaultdict(Counter)
rows = []
responses_by_sent_by_model = {model: defaultdict(list) for model in model_dict}

for i in df.index:
    worker = df.loc[i, 'WorkerId']
    worker_id = 'W' + str(worker_dict[worker])
    for j in range(25):
        model = df.loc[i, 'Input.TYPE' + str(j)]        
        ctx = df.loc[i, 'Input.CTX' + str(j)]
        item_id = 'I' + str(ctx_dict[ctx])
        output = df.loc[i, 'Input.OUT' + str(j)]
        response = df.loc[i, 'Answer.' + str(j)]
        response_val = float(response_dict[response])
        rows.append([worker, worker_id, model, ctx, item_id, output, response, response_val])
        responses_by_sent_by_model[model][output].append(response)

df_out = pd.DataFrame(rows, columns=['worker', 'worker_id', 'model', 'input', 'input_id', 'output', 'response', 'response_val'])
print(df_out.shape)
df_out.head()

(4000, 8)


,worker,worker_id,model,input,input_id,output,response,response_val
0,AFUO1VYEFRMO4,W108,LM,0,I0,London attended as their candidate for the 201...,Specific,2.0
1,AFUO1VYEFRMO4,W108,LM,1,I1,"NOBODY doubts that spring is here , starting w...",Specific,2.0
2,AFUO1VYEFRMO4,W108,LM,2,I2,' If you want problems with our phone service ...,Invalid,0.0
3,AFUO1VYEFRMO4,W108,LM,3,I3,Mr Marchesani and his family set up a small bu...,Invalid,0.0
4,AFUO1VYEFRMO4,W108,LM,4,I4,The White House said it is aware of the incide...,Very Typical,5.0


In [10]:
print(set(df_out['model'].values))

{'HUMAN_LM', 'LM'}


In [11]:
model_code_map = {'HUMAN_LM': 0.5, 'LM': -0.5}
model_codes = [model_code_map[m] for m in df_out['model'].values]
df_out['model_code'] = model_codes
df_out.head()

,worker,worker_id,model,input,input_id,output,response,response_val,model_code
0,AFUO1VYEFRMO4,W108,LM,0,I0,London attended as their candidate for the 201...,Specific,2.0,-0.5
1,AFUO1VYEFRMO4,W108,LM,1,I1,"NOBODY doubts that spring is here , starting w...",Specific,2.0,-0.5
2,AFUO1VYEFRMO4,W108,LM,2,I2,' If you want problems with our phone service ...,Invalid,0.0,-0.5
3,AFUO1VYEFRMO4,W108,LM,3,I3,Mr Marchesani and his family set up a small bu...,Invalid,0.0,-0.5
4,AFUO1VYEFRMO4,W108,LM,4,I4,The White House said it is aware of the incide...,Very Typical,5.0,-0.5


In [12]:
outdir = os.path.join('..', 'data', 'annotated_datasets')
if not os.path.exists(outdir):
    os.makedirs(outdir)

df_out.to_csv(os.path.join(outdir, 'huse_lm.csv'))